# Optimization Models

Before creating our modified TSP model, let's analyze the existing Gurobi code for the TSP to understand the changes we'll need to make. We'll start by importing our necessary Python modules and libraries as well as the data we'll use to solve our models.

In [ ]:
# Importing necessary libraries and modules

%pip install gurobipy                 # Gurobi is our optimization engine
import gurobipy as gp
from gurobipy import GRB

from itertools import combinations    # The combintions module is used to create our city pairs
import math                           # The math library is used for calculating the distance between our cities
import folium                         # The folium library is used to create our map to visualize our result
from IPython.display import display   # The display library is used to display the map

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.9 MB/s eta 0:00:00


In [ ]:
# Making list of F1 cities and dictionary of city coordinates

# Initializing F1 venues as a list of dictionaries
f1_venues =  [
    {"city": "Yas Marina", "country": "Abu Dhabi", "latitude": 24.4672, "longitude": 54.6037},
    {"city": "Melbourne", "country": "Australia", "latitude": -37.8136, "longitude": 144.9631},
    {"city": "Shanghai", "country": "China", "latitude": 31.2304, "longitude": 121.4737},
    {"city": "Sakhir", "country": "Bahrain", "latitude": 26.0325, "longitude": 50.5106},
    {"city": "Jeddah", "country": "Saudi Arabia", "latitude": 21.4858, "longitude": 39.1925},
    {"city": "Miami", "country": "USA", "latitude": 25.7617, "longitude": -80.1918},
    {"city": "Imola", "country": "Italy", "latitude": 44.3559, "longitude": 11.7161},
    {"city": "Monaco", "country": "Monaco", "latitude": 43.7384, "longitude": 7.4246},
    {"city": "São Paulo", "country": "Brazil", "latitude": -23.5505, "longitude": -46.6333},
    {"city": "Barcelona", "country": "Spain", "latitude": 41.3851, "longitude": 2.1734},
    {"city": "Montreal", "country": "Canada", "latitude": 45.5017, "longitude": -73.5673},
    {"city": "Spielberg", "country": "Austria", "latitude": 47.2195, "longitude": 14.7799},
    {"city": "Silverstone", "country": "UK", "latitude": 52.0742, "longitude": -1.0141},
    {"city": "Spa", "country": "Belgium", "latitude": 50.4921, "longitude": 5.8641},
    {"city": "Budapest", "country": "Hungary", "latitude": 47.4979, "longitude": 19.0402},
    {"city": "Zandvoort", "country": "Netherlands", "latitude": 52.3840, "longitude": 4.5294},
    {"city": "Suzuka", "country": "Japan", "latitude": 34.8823, "longitude": 136.5847},
    {"city": "Monza", "country": "Italy", "latitude": 45.5845, "longitude": 9.2744},
    {"city": "Singapore", "country": "Singapore", "latitude": 1.3521, "longitude": 103.8198},
    {"city": "Baku", "country": "Azerbaijan", "latitude": 40.4093, "longitude": 49.8671},
    {"city": "Austin", "country": "USA", "latitude": 30.2672, "longitude": -97.7431},
    {"city": "Mexico City", "country": "Mexico", "latitude": 19.4326, "longitude": -99.1332},
    {"city": "Las Vegas", "country": "USA", "latitude": 36.1699, "longitude": -115.1398},
    {"city": "Lusail", "country": "Qatar", "latitude": 25.4670, "longitude": 51.3960}
]

# Adding the cities to a list and the coordinates to a dictionary
cities = []
coordinates = {}
for venue in f1_venues:
    city = venue['city']
    cities.append(city)
    coordinates[city] = (float(venue['latitude']), float(venue['longitude']))

In [ ]:
# Calculating distances in kilometers between each city pair using the Haversine formula

# Function to calculate distances
def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]

    # Converting latitude and longitude from degrees to radians
    lat1 = math.radians(c1[0])
    lon1 = math.radians(c1[1])
    lat2 = math.radians(c2[0])
    lon2 = math.radians(c2[1])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers
    R = 6371.0
    distance = R * c
    return distance

# Dictionary comprehension to calculate the distance in kilometers between every city pair
dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(cities, 2)}

### Gurobi TSP Model

Now we'll analyze the Gurobi TSP model. Link: https://www.gurobi.com/jupyter_models/traveling-salesman/

In [ ]:
# Callback function - use lazy constraints to eliminate sub-tours
def subtourelim_tsp(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour_tsp(selected)
        if len(tour) < len(cities):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)

# Function to find the shortest subtour given a tuplelist of edges
def subtour_tsp(edges):
    unvisited = cities[:]
    cycle = cities[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

# Function to create and solve model
def tsp_model():
    # Model creation
    m = gp.Model()

    # Variables: is city 'i' adjacent to city 'j' on the tour?
    vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

    # Symmetric direction: use dict.update to alias variable with new key
    vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

    # Constraints: two edges incident to each city
    cons = m.addConstrs(vars.sum(c, '*') == 2 for c in cities)

    # Model solving
    m._vars = vars
    m.Params.lazyConstraints = 1
    m.setParam('OutputFlag', 0)
    m.optimize(subtourelim_tsp)

    return m.getAttr('x', vars), m.ObjVal

The key aspects of this model we want to change are the process used to eliminate subtours and the constraints used to define how many edges a city can have.



*   For the subtour elimination process, we want to ensure subtours are eliminated but not sub-paths. A sub-path would be a tour that doesn't return to the original city, but doesn't visit all the cities.
*   For the number of edges, we want to change the constraint to allow each city to have either 1 or 2 edges. But, we also want to make sure the total number of edges are n-1, as this will ensure a path is created.



To wrap up this analysis, we'll retrieve the solution for this model and map it using Folium.

In [ ]:
def solve_tsp():
    # Retrieving solution edges and creating our tour from it
    solved_vars, solved_obj = tsp_model()
    selected = gp.tuplelist((i, j) for i, j in solved_vars.keys() if solved_vars[i, j] > 0.5)
    tour = subtour_tsp(selected)
    assert len(tour) == len(cities)

    # Printing our tour and the length of it
    print(f"Our tour is: {tour}")
    print(f"The length of our tour is: {solved_obj:.2f} km")

    # Mapping tour
    map = folium.Map(location=[0, 0], zoom_start=2, min_zoom=2, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in tour:
        points.append(coordinates[city])
        folium.CircleMarker(
            location=coordinates[city],
            radius=5,
            color='red',
            fill=True,
            fill_color='white',
            tooltip=city
        ).add_to(map)
    points.append(points[0])
    folium.PolyLine(points, color='black').add_to(map)
    display(map)

solve_tsp()

Set parameter LazyConstraints to value 1
Our tour is: ['Yas Marina', 'Baku', 'Suzuka', 'Shanghai', 'Singapore', 'Melbourne', 'São Paulo', 'Miami', 'Mexico City', 'Austin', 'Las Vegas', 'Montreal', 'Silverstone', 'Zandvoort', 'Spa', 'Barcelona', 'Monaco', 'Monza', 'Imola', 'Spielberg', 'Budapest', 'Jeddah', 'Sakhir', 'Lusail']
The length of our tour is: 62404.34 km


### Freely Open Loop TSP Model

Now we'll create our new TSP model by modifying the above code to find the shortest path instead of the shortest cycle. For brevity, I've only added comments for the main changes in the code, but you can find a detailed explaination of the model in the Appendix.

In [ ]:
# Function to check any feasible solutions for subtours (callback function)
def subtourelim_fol_tsp(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] != 0)
        tour = subtour_fol_tsp(selected)
        if len(tour) < len(cities):
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)

# Function to find the path and shortest subtour in our list of edges
def subtour_fol_tsp(edges):
    unvisited = cities[:]
    cycle = cities[:]
    path = []   # variable to store any identified paths as to not eliminate them
    while unvisited:
        thistour = []
        neighbors = unvisited
        while neighbors:
            cycle_found = 0   # variable to indicate if a cycle is found
            current = neighbors[0]
            thistour.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current, '*') if j in unvisited]

            # Process to check if we found a cycle or the end of a path when no more neighbors are found
            if not neighbors:
                if thistour[0] in [j for i,j in edges.select(current, '*')] and len(thistour) > 2:
                    cycle_found = 1
                    if len(thistour) <= len(cycle):
                        cycle = thistour
                    break
                thistour.reverse()
                current = thistour[-1]
                neighbors = [i for i,j in edges.select('*', current) if i in unvisited]

        # If we reach the end of the tour after looking at both ends, we then set this to be the path we found
        if not cycle_found:
            path = thistour

    # If the path we found contains all the cities, then we return the path (valid solution)
    if len(path) == len(cities):
        return path

    # If not, then we return the shortest cycle to be eliminated
    return cycle

# Function to create and solve model
def fol_tsp_model(start_city, end_city):
    m = gp.Model()

    # Decision variables / objective coefficient
    vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')
    vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

    # Constraints
    if start_city:
        m.addConstr(vars.sum(start_city, '*') == 1)
    if end_city:
        m.addConstr(vars.sum(end_city, '*') == 1)

    # The number of edges coming out of a city has to be 1 or 2
    m.addConstrs(vars.sum(c, '*') <= 2 for c in cities)
    m.addConstrs(vars.sum(c, '*') >= 1 for c in cities)

    # There must be n-1 edges total (this ensures a path)
    m.addConstr((gp.quicksum(vars[(i,j)] for i,j in dist.keys()) == len(cities)-1))

    # Model solving
    m.setParam('OutputFlag', 0)
    m._vars = vars
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim_fol_tsp)

    return m.getAttr('x', vars), m.ObjVal

First, we'll solve our model without any start or end city constraints. To make this process easy, we'll create a function to solve the model based on specified start cities and end cities.

In [ ]:
# Function to solve fol_tsp model under different start and end city conditions
def solve_fol_tsp(start_city = "", end_city = "", context = "F1"):
    # Retrieving solution edges and creating our path from it
    solved_vars, solved_obj = fol_tsp_model(start_city, end_city)
    selected = gp.tuplelist((i, j) for i, j in solved_vars.keys() if solved_vars[i, j] != 0)
    tour = subtour_fol_tsp(selected)
    assert len(tour) == len(cities)
    if start_city:
        if tour[0] != start_city:
            tour = tour[::-1]
    if end_city:
        if tour[-1] != end_city:
            tour = tour[::-1]

    # Printing our tour and the length of it
    print(f"Our tour is: {tour}")
    print(f"The length of our tour is {solved_obj:.2f} kilometers")

    # Mapping tour
    if context == "F1":
        map = folium.Map(location=[0, 0], zoom_start=2, min_zoom=2, max_bounds=True, tiles="CartoDB positron")
    elif context == "US":
        map = folium.Map(location=[40, -95], zoom_start=4, min_zoom=4, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in tour:
        points.append(coordinates[city])
        if city == start_city:
            folium.Marker(
                location=coordinates[city],
                tooltip=city,
                icon=folium.Icon(icon='star',color='green')
            ).add_to(map)
        elif city == end_city:
            folium.Marker(
                location=coordinates[city],
                tooltip=city,
                icon=folium.Icon(icon='star',color='red')
            ).add_to(map)
        else:
            folium.CircleMarker(
                location=coordinates[city],
                radius=5,
                color='red',
                fill=True,
                fill_color='white',
                tooltip=city
            ).add_to(map)
    folium.PolyLine(points, color='black').add_to(map)
    display(map)

solve_fol_tsp()

Our tour is: ['São Paulo', 'Miami', 'Mexico City', 'Austin', 'Las Vegas', 'Montreal', 'Silverstone', 'Zandvoort', 'Spa', 'Barcelona', 'Monaco', 'Monza', 'Imola', 'Spielberg', 'Budapest', 'Jeddah', 'Sakhir', 'Lusail', 'Yas Marina', 'Baku', 'Suzuka', 'Shanghai', 'Singapore', 'Melbourne']
The length of our tour is 49319.45 kilometers


From this initial solve, we see that our objective value is indeed less than that of the regular TSP code (if it wasn't then there would be something very wrong with our code). Looking closer though, it seems like our shortest path is just the shortest tour minues the longest edge; we'll analyze this relationship later.

Next, we'll solve our model under different combinations of start and end cities. It should be noted that in the case of just a start city or just an end city, the path will effectively be the same if the start city was the end city and vice versa.

In [ ]:
# Initializing test cases
test_cases = [("Yas Marina", ""), ("", "Suzuka"), ("Monaco", "Las Vegas"), ("Spa", "Monza"), ("Melbourne", "Yas Marina")]

# Solving different test cases
for city_pair in test_cases:
    start, end = city_pair
    print("Case:")
    print(f"\tStart city: {start}")
    print(f"\tEnd city: {end}")
    solve_fol_tsp(start, end)

Case:
	Start city: Yas Marina
	End city: 
Our tour is: ['Yas Marina', 'Lusail', 'Sakhir', 'Jeddah', 'Baku', 'Budapest', 'Spielberg', 'Imola', 'Monza', 'Monaco', 'Barcelona', 'Spa', 'Zandvoort', 'Silverstone', 'Montreal', 'Las Vegas', 'Austin', 'Mexico City', 'Miami', 'São Paulo', 'Melbourne', 'Singapore', 'Shanghai', 'Suzuka']
The length of our tour is 54731.89 kilometers


Case:
	Start city: 
	End city: Suzuka
Our tour is: ['São Paulo', 'Miami', 'Mexico City', 'Austin', 'Las Vegas', 'Montreal', 'Silverstone', 'Zandvoort', 'Spa', 'Barcelona', 'Monaco', 'Monza', 'Imola', 'Spielberg', 'Budapest', 'Baku', 'Jeddah', 'Sakhir', 'Lusail', 'Yas Marina', 'Singapore', 'Melbourne', 'Shanghai', 'Suzuka']
The length of our tour is 51771.07 kilometers


Case:
	Start city: Monaco
	End city: Las Vegas
Our tour is: ['Monaco', 'Barcelona', 'Silverstone', 'Zandvoort', 'Spa', 'Monza', 'Imola', 'Spielberg', 'Budapest', 'Jeddah', 'Sakhir', 'Lusail', 'Yas Marina', 'Baku', 'Suzuka', 'Shanghai', 'Singapore', 'Melbourne', 'São Paulo', 'Miami', 'Montreal', 'Austin', 'Mexico City', 'Las Vegas']
The length of our tour is 57762.52 kilometers


Case:
	Start city: Spa
	End city: Monza
Our tour is: ['Spa', 'Zandvoort', 'Silverstone', 'Montreal', 'Las Vegas', 'Austin', 'Mexico City', 'Miami', 'São Paulo', 'Melbourne', 'Singapore', 'Shanghai', 'Suzuka', 'Baku', 'Yas Marina', 'Lusail', 'Sakhir', 'Jeddah', 'Budapest', 'Spielberg', 'Imola', 'Monaco', 'Barcelona', 'Monza']
The length of our tour is 61953.18 kilometers


Case:
	Start city: Melbourne
	End city: Yas Marina
Our tour is: ['Melbourne', 'Singapore', 'Shanghai', 'Suzuka', 'Las Vegas', 'Mexico City', 'Austin', 'Montreal', 'Miami', 'São Paulo', 'Barcelona', 'Silverstone', 'Zandvoort', 'Spa', 'Monza', 'Monaco', 'Imola', 'Spielberg', 'Budapest', 'Baku', 'Jeddah', 'Sakhir', 'Lusail', 'Yas Marina']
The length of our tour is 54843.18 kilometers


Through these examples, we've effectively shown that our new Freely-Open-Loop TSP model can find the shortest path among our F1 cities with and without start and end city conditions.

# Solution Validation

Now that we've created a working Freely-Open-Loop TSP model for our problem, we'll compare it to a few heuristic solutions to confirm we have found a better objective value. We will do this by solving our model and comparison models using US capitals data and without any start or end city constraints.

We'll start by importing our US capital data from the same source as the original TSP Gurobi code. We'll also create our cities list, coordinates dictionary, and distances dictionary.

In [ ]:
# Bringing in US capital data

import json                           # The json library is used to read in the json file with the capitals data
import urllib.request                 # The urllib.request library is used to access the capitals data json file

# Accessing capitals data and reading in as a json file
url = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/traveling_salesman/capitals.json'
data = urllib.request.urlopen(url).read()
capitals_json = json.loads(data)

# Adding the capitals to a list and coordinates to a dictionary
cities = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']:
      capital = capitals_json[state]['capital']
      cities.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

# Calculating distances in kilometers between each city pair using the Haversine formula
def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]

    # Converting latitude and longitude from degrees to radians
    lat1 = math.radians(c1[0])
    lon1 = math.radians(c1[1])
    lat2 = math.radians(c2[0])
    lon2 = math.radians(c2[1])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers
    R = 6371.0
    distance = R * c
    return distance

# Dictionary comprehension to calculate the distance in kilometers between every city pair
dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(cities, 2)}

Next we'll solve our Freely-Open-Loop TSP model using this data.

In [ ]:
solve_fol_tsp(context="US")

Our tour is: ['Olympia', 'Salem', 'Sacramento', 'Carson City', 'Boise', 'Helana', 'Salt Lake City', 'Phoenix', 'Santa Fe', 'Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'Saint Paul', 'Madison', 'Des Moines', 'Lincoln', 'Topeka', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Lansing', 'Columbus', 'Charleston', 'Frankfort', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Richmond', 'Annapolis', 'Dover', 'Harrisburg', 'Trenton', 'Albany', 'Hartford', 'Providence', 'Boston', 'Concord', 'Montpelier', 'Augusta']
The length of our tour is 15709.00 kilometers


Now we will solve the problem using the following heuristic models:


*   Nearest Neighbor Method
*   Nearest Insertion Method
*   TSP Minus Longest Edge Method



### Nearest Neighbor Method

The nearest neighbor method works by starting at a specified city then visiting the closest city to that city. Then you visit the next closest city to that city and so on until all the cities are visited. Since we aren't having a starting city constraint, we'll repeat this algorithm for each city as the starting city and then choosing the shortest path as our solution.

In [ ]:
# Function to find the nearest neighbor to a city
def nearest_neighbor(current_city, unvisited_cities):
    min_distance = float('inf')
    nearest_neighbor = None
    for city in unvisited_cities:
        try:
            d = dist[current_city, city]
        except:
            d = dist[city, current_city]
        if d < min_distance:
            min_distance = d
            nearest_neighbor = city
    return nearest_neighbor

# Function that runs the nearest neighbor algorithm
def nearest_neighbor_algorithm(start_city):
    total_distance = 0
    unvisited = cities[:]
    current_city = start_city
    path = [current_city]
    unvisited.remove(current_city)
    while unvisited:
        next_city = nearest_neighbor(current_city, unvisited)
        path.append(next_city)
        unvisited.remove(next_city)
        try:
            total_distance += dist[current_city, next_city]
        except:
            total_distance += dist[next_city, current_city]
        current_city = next_city
    return path, total_distance

# Function that runs model
def nn_model(start_city=""):
    if start_city:
        return nearest_neighbor_algorithm(start_city)
    else:
        paths = []
        for city in cities:
            paths.append(nearest_neighbor_algorithm(city))
        paths.sort(key = lambda x: x[1])
        return paths[0]

def solve_nn_model(start_city=""):
    # Getting our path and total_distance from our solution
    path, total_distance = nn_model()

    # Printing tour
    print(f"Our tour is: {path}")
    print(f"The length of our tour is {total_distance:.2f} kilometers")

    # Mapping tour
    map_nn = folium.Map(location=[40, -95], zoom_start=4, min_zoom=4, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in path:
        points.append(coordinates[city])
        folium.CircleMarker(
            location=coordinates[city],
            radius=5,
            color='red',
            fill=True,
            fill_color='white',
            tooltip=city
        ).add_to(map_nn)
    folium.PolyLine(points, color='black').add_to(map_nn)
    display(map_nn)

solve_nn_model()

Our tour is: ['Augusta', 'Concord', 'Boston', 'Providence', 'Hartford', 'Albany', 'Montpelier', 'Trenton', 'Dover', 'Annapolis', 'Harrisburg', 'Richmond', 'Raleigh', 'Columbia', 'Atlanta', 'Montgomery', 'Tallahassee', 'Jackson', 'Baton Rouge', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Frankfort', 'Columbus', 'Charleston', 'Nashville', 'Lansing', 'Madison', 'Saint Paul', 'Des Moines', 'Lincoln', 'Topeka', 'Oklahoma City', 'Austin', 'Santa Fe', 'Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'Helana', 'Boise', 'Salt Lake City', 'Carson City', 'Sacramento', 'Salem', 'Olympia', 'Phoenix']
The length of our tour is 17943.11 kilometers


### Nearest Insertion Method

The nearest insertion method works is siilar to the nearest neighbor method, but rather than just checking the nearest city to the most recently added city, it checks the nearest city to the current edges of the path and goes to the closest one from those two. Similarly, this process repeats until all cities are visited, and we'll also cycle through each city as the starting city and choose the shortest overall path as our solution.

In [ ]:
# Function to find the nearest neighbor to a city
def nearest_neighbor(current_city, unvisited_cities):
    min_distance = float('inf')
    nearest_neighbor = None
    for city in unvisited_cities:
        try:
            d = dist[current_city, city]
        except:
            d = dist[city, current_city]
        if d < min_distance:
            min_distance = d
            nearest_neighbor = city
    return nearest_neighbor, min_distance

# Function that runs the nearest neighbor algorithm
def nearest_insertion_algorithm(start_city):
    total_distance = 0
    unvisited = cities[:]
    path = [start_city]
    unvisited.remove(start_city)
    while unvisited:
        city_1 = path[0]
        city_2 = path[-1]
        next_city_1, dist_1 = nearest_neighbor(city_1, unvisited)
        next_city_2, dist_2 = nearest_neighbor(city_2, unvisited)
        if dist_1 <= dist_2:
            path = [next_city_1] + path
            total_distance += dist_1
            unvisited.remove(next_city_1)
        else:
            path = path + [next_city_2]
            total_distance += dist_2
            unvisited.remove(next_city_2)
    return path, total_distance

# Function that runs model
def ni_model(start_city=""):
    if start_city:
        return nearest_insertion_algorithm(start_city)
    else:
        paths = []
        for city in cities:
            paths.append(nearest_insertion_algorithm(city))
        paths.sort(key = lambda x: x[1])
        return paths[0]

def solve_ni_model(start_city=""):
    # Getting our path and total_distance from our solution
    path, total_distance = ni_model()

    # Printing tour
    print(f"Our tour is: {path}")
    print(f"The length of our tour is {total_distance:.2f} kilometers")

    # Mapping tour
    map_ni = folium.Map(location=[40, -95], zoom_start=4, min_zoom=4, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in path:
        points.append(coordinates[city])
        folium.CircleMarker(
            location=coordinates[city],
            radius=5,
            color='red',
            fill=True,
            fill_color='white',
            tooltip=city
        ).add_to(map_ni)
    folium.PolyLine(points, color='black').add_to(map_ni)
    display(map_ni)

solve_ni_model()

Our tour is: ['Augusta', 'Montpelier', 'Concord', 'Boston', 'Providence', 'Hartford', 'Albany', 'Harrisburg', 'Trenton', 'Dover', 'Annapolis', 'Richmond', 'Charleston', 'Columbus', 'Frankfort', 'Indianapolis', 'Springfield', 'Jefferson City', 'Des Moines', 'Lincoln', 'Topeka', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Lansing', 'Madison', 'Saint Paul', 'Pierre', 'Bismarck', 'Cheyenne', 'Denver', 'Santa Fe', 'Phoenix', 'Salt Lake City', 'Boise', 'Helana', 'Olympia', 'Salem', 'Carson City', 'Sacramento']
The length of our tour is 16719.53 kilometers


### TSP Minus Longest Edge Method

This method goes back to the callout we made earlier for how the shortest path we found using optimization techniques was the same as the shortest cycle without the longest edge. To test whether our path solution just gives the cycle solution minus the longest edge, we'll run the regular original TSP algorithm, remove the longest edge, and then determine the path from it as our solution.

In [ ]:
def solve_tsp_minus():
    # Retrieving solution from original tsp model code
    solved_vars, solved_obj = tsp_model()
    selected = gp.tuplelist((i, j) for i, j in solved_vars.keys() if solved_vars[i, j] > 0.5)

    # Remove longest edge from solution
    pair_distances = []
    for pair in selected:
        try:
            d = dist[pair]
        except:
            d = dist[pair[::-1]]
        pair_distances.append((pair, d))
    pair_distances.sort(key=lambda x: x[1])
    longest_distance = pair_distances[-1][1]
    pair_distances.pop()
    pair_distances.pop()
    selected = gp.tuplelist(pair for pair, dist in pair_distances)

    # Find tour and confirm it works
    tour = subtour_fol_tsp(selected)
    assert len(tour) == len(cities)

    # Printing our tour and length
    print(f"Our tour is: {tour}")
    print(f"The length of our tour is {(solved_obj-longest_distance):.2f} kilometers")

    # Mapping tour
    map_tsp_minus = folium.Map(location=[40, -95], zoom_start=4, min_zoom=4, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in tour:
        points.append(coordinates[city])
        folium.CircleMarker(
            location=coordinates[city],
            radius=5,
            color='red',
            fill=True,
            fill_color='white',
            tooltip=city
        ).add_to(map_tsp_minus)
    folium.PolyLine(points, color='black').add_to(map_tsp_minus)
    display(map_tsp_minus)

solve_tsp_minus()

Set parameter LazyConstraints to value 1
Our tour is: ['Carson City', 'Sacramento', 'Salem', 'Olympia', 'Boise', 'Helana', 'Salt Lake City', 'Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'Saint Paul', 'Madison', 'Lansing', 'Columbus', 'Charleston', 'Harrisburg', 'Albany', 'Montpelier', 'Augusta', 'Concord', 'Boston', 'Providence', 'Hartford', 'Trenton', 'Dover', 'Annapolis', 'Richmond', 'Raleigh', 'Columbia', 'Tallahassee', 'Montgomery', 'Atlanta', 'Nashville', 'Frankfort', 'Indianapolis', 'Springfield', 'Des Moines', 'Lincoln', 'Topeka', 'Jefferson City', 'Little Rock', 'Jackson', 'Baton Rouge', 'Austin', 'Oklahoma City', 'Santa Fe', 'Phoenix']
The length of our tour is 16156.23 kilometers


### Summary of Comparisons

The table below summarizes the results of our comparisons. We've successfully shown that our method beats the heurestic methods.

| Method | Distance (km) |
|--------|---------------|
| Freely-Open-Loop TSP | 15,709 |
| Nearest Neighbor Method | 17,943.11 |
| Nearest Insertion Method | 16,719.53 |
| TSP Minus Longest Edge Method | 16,156.23 |



*   What did I accomplish
*   Webapp link
*   What did I learn
*   What are the limitations and potential future improvements



# Appendix

## Freely-Open Loop TSP Model Code

In [ ]:
# Function to check any feasible solutions for subtours (callback function)
def subtourelim_fol_tsp(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if round(vals[i, j]) != 0)
        tour = subtour_fol_tsp(selected)
        if len(tour) < len(cities):
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)

# Function to find the path and shortest subtour in our list of edges
def subtour_fol_tsp(edges):
    unvisited = cities[:]
    cycle = cities[:]
    path = []   # variable to store any identified paths as to not eliminate them
    while unvisited:
        thistour = []
        neighbors = unvisited
        while neighbors:
            cycle_found = 0   # variable to indicate if a cycle is found
            current = neighbors[0]
            thistour.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current, '*') if j in unvisited]

            # Process to check if we found a cycle or the end of a path when no more neighbors are found
            if not neighbors:
                if thistour[0] in [j for i,j in edges.select(current, '*')] and len(thistour) > 2:
                    cycle_found = 1
                    if len(thistour) <= len(cycle):
                        cycle = thistour
                    break
                thistour.reverse()
                current = thistour[-1]
                neighbors = [i for i,j in edges.select('*', current) if i in unvisited]

        # If we reach the end of the tour after looking at both ends, we then set this to be the path we found
        if not cycle_found:
            path = thistour

    # If the path we found contains all the cities, then we return the path (valid solution)
    if len(path) == len(cities):
        return path

    # If not, then we return the shortest cycle to be eliminated
    return cycle

# Function to create and solve model
def fol_tsp_model(start_city, end_city):
    m = gp.Model()

    # Decision variables / objective coefficient
    vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')
    vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

    # Constraints
    if start_city:
        m.addConstr(vars.sum(start_city, '*') == 1)
    if end_city:
        m.addConstr(vars.sum(end_city, '*') == 1)

    # The number of edges coming out of a city has to be 1 or 2
    m.addConstrs(vars.sum(c, '*') <= 2 for c in cities)
    m.addConstrs(vars.sum(c, '*') >= 1 for c in cities)

    # There must be n-1 edges total (this ensures a path)
    m.addConstr((gp.quicksum(vars[(i,j)] for i,j in dist.keys()) == len(cities)-1))

    # Model solving
    m.setParam('OutputFlag', 0)
    m._vars = vars
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim_fol_tsp)

    return m.getAttr('x', vars), m.ObjVal

# Function to solve fol_tsp model under different start and end city conditions
def solve_fol_tsp(start_city = "", end_city = "", context = "F1"):
    # Retrieving solution edges and creating our path from it
    solved_vars, solved_obj = fol_tsp_model(start_city, end_city)
    selected = gp.tuplelist((i, j) for i, j in solved_vars.keys() if round(solved_vars[i, j]) != 0)
    tour = subtour_fol_tsp(selected)
    assert len(tour) == len(cities)
    if start_city:
        if tour[0] != start_city:
            tour = tour[::-1]
    if end_city:
        if tour[-1] != end_city:
            tour = tour[::-1]

    # Printing our tour and the length of it
    print(f"Our tour is: {tour}")
    print(f"The length of our tour is {solved_obj:.2f} kilometers")

    # Mapping tour
    if context == "F1":
        map = folium.Map(location=[0, 0], zoom_start=2, min_zoom=2, max_bounds=True, tiles="CartoDB positron")
    elif context == "US":
        map = folium.Map(location=[40, -95], zoom_start=4, min_zoom=4, max_bounds=True, tiles="CartoDB positron")
    points = []
    for city in tour:
        points.append(coordinates[city])
        if city == start_city:
            folium.Marker(
                location=coordinates[city],
                tooltip=city,
                icon=folium.Icon(icon='star',color='green')
            ).add_to(map)
        elif city == end_city:
            folium.Marker(
                location=coordinates[city],
                tooltip=city,
                icon=folium.Icon(icon='star',color='red')
            ).add_to(map)
        else:
            folium.CircleMarker(
                location=coordinates[city],
                radius=5,
                color='red',
                fill=True,
                fill_color='white',
                tooltip=city
            ).add_to(map)
    folium.PolyLine(points, color='black').add_to(map)
    display(map)

solve_fol_tsp()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 97 rows, 1128 columns and 5640 nonzeros
Model fingerprint: 0xd14bf2db
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve time: 0.01s
Presolved: 97 rows, 1128 columns, 5640 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 1.516854e+04, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15168.5435    0   12          - 1516